In [ ]:
!pip install kaggle kagglehub opencv-python-headless scikit-learn pandas seaborn matplotlib

In [ ]:
import os
from google.colab import userdata

os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

# Create the config folder and file so the kaggle CLI works
!mkdir -p ~/.kaggle
!echo '{"username":"'$KAGGLE_USERNAME'","key":"'$KAGGLE_KEY'"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

print("Successfully configured Kaggle!")

In [ ]:
import os
import cv2
import numpy
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    precision_score, recall_score, f1_score, confusion_matrix,
    classification_report
)

def load_and_preprocess_data(base_path, categories, img_size=(64, 64)):
    data, labels = [], []
    for folder_name, label in categories.items():
        category_path = os.path.join(base_path, folder_name)
        if not os.path.isdir(category_path):
            continue

        for img_name in os.listdir(category_path):
            # Skip system files and only keep images (skip thumb.db etc.)
            if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue

            img_path = os.path.join(category_path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                # Resize to a standard size
                img = cv2.resize(img, img_size)
                # Normalize pixels to range [0, 1]
                data.append(img.astype('float32') / 255.0)
                labels.append(label)

    return numpy.array(data), numpy.array(labels)

def evaluate_model(model, X_test, y_test, class_names=None):
    predictions = model.predict(X_test)
    accuracy = (numpy.sum(predictions == y_test) / len(y_test)) * 100

    # Precision: correctly predicted / the total predicted
    # High precision means fewer false positives.
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    # Recall:  correctly predicted / all observations in actual class
    # High recall means fewer false negatives.
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    # F1-Score: The weighted average of Precision and Recall. It tries to find the balance between precision and recall
    # A high F1-score indicates good performance for both precision and recall
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    # Confusion Matrix: A table that shows the counts of true positives, true negatives, false positives, and false negatives
    cm = confusion_matrix(y_test, predictions)
    # Mean Absolute Error (MAE): The average of the absolute differences between predictions and actual values
    # It gives an idea of the average magnitude of errors.
    mae = mean_absolute_error(y_test, predictions)
    # Root Mean Squared Error (RMSE): The square root of the average of the squared differences between predictions and actual values
    # average distance from correct predicton
    rmse = numpy.sqrt(mean_squared_error(y_test, predictions))
    # R-squared (R2 Score): Represents the proportion of the variance in the dependent variable that is predictable
    # 1 means model understands the class perfectly 0 means model is no better than random guessing - means model does not capture any of the variance in the data
    r2 = r2_score(y_test, predictions)

    results = {
        "model":     model.name,
        "accuracy":  round(accuracy, 4),
        "precision": round(precision, 4),
        "recall":    round(recall, 4),
        "f1":        round(f1, 4),
        "mae":       round(mae, 4),
        "rmse":      round(rmse, 4),
        "r2":        round(r2, 4),
        "confusion_matrix": cm.tolist()
    }

    print(f"\n{'='*50}")
    print(f"Results for: {model.name}")
    print(f"  Accuracy:  {accuracy:.2f}%")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")
    print(f"  MAE:       {mae:.4f}")
    print(f"  RMSE:      {rmse:.4f}")
    print(f"  R\u00b2:        {r2:.4f}")
    if class_names:
        print("\nClassification Report:")
        print(classification_report(y_test, predictions, target_names=class_names))
    print(f"Confusion Matrix:\n{cm}")

    return results

def clean_nan_values(data_input):
    nan_representations = ['--', 'na', 'n/a', 'nan', 'none', '']

    data_array = numpy.asarray(data_input, dtype=object)
    cleaned_array = numpy.copy(data_array)

    # Iterate through each element of the array
    it = numpy.nditer(data_array, flags=['multi_index', 'refs_ok'])
    for x in it:
        idx = it.multi_index
        value = x.item()  # Get the actual value from array

        if isinstance(value, str):
            # If the value is a string, check if its lowercase version matches any NaN representation
            if value.lower() in nan_representations:
                cleaned_array[idx] = numpy.nan
        elif value is None:
            # Explicitly treat Python's None as a NaN
            cleaned_array[idx] = numpy.nan

    # Attempt to convert the cleaned array to a floating-point numeric type
    try:
        temp_numeric_array = cleaned_array.astype(float)
        cleaned_array = temp_numeric_array
    except ValueError:
        print("Warning: Some values could not be converted to numeric type after NaN cleaning. "
              "The array will remain of object dtype to preserve data integrity.")

    return cleaned_array

print("utils.py loaded successfully.")

In [ ]:
# Algorythm 1 Euclidean Distance: looking at nearest neighbour and predicting as that class
class EuclideanDistance:
    def __init__(self):
        self.X_train = None
        self.y_train = None
        self.name = "Euclidean Distance"

    def train(self, X_train, y_train):
        print(f"\nTraining {self.name}...")
        self.X_train = X_train
        self.y_train = y_train
        print(f"Memorized {len(X_train)} training examples")

    def predict_single(self, x):
        # Calculate Euclidean distance to all training samples
        # Distance formula: sqrt(sum((x - train_sample)^2))
        distances = numpy.sqrt(numpy.sum((self.X_train - x)**2, axis=1))

        # Find the index of the smallest distance
        closest_index = numpy.argmin(distances)

        # Return the label of the closest training sample
        return self.y_train[closest_index]

    def predict(self, X_test):
        predictions = []
        for x in X_test:
            predictions.append(self.predict_single(x))
        return numpy.array(predictions)


# Algorythm 2 K Nearest Neighbours: looking at k nearest neighbours and predicting as most popular class among them
class KNearestNeighbors:
    def __init__(self, k=3):
        # k: Number of neighbors to vote (default 3)
        self.k = k
        self.X_train = None
        self.y_train = None
        self.name = f"K-Nearest Neighbors (k={k})"

    def train(self, X_train, y_train):
        print(f"\nTraining {self.name}...")
        self.X_train = numpy.array(X_train)
        self.y_train = numpy.array(y_train)
        print(f"Memorized {len(X_train)} training examples")

    def predict_single(self, x):
        # Calculate distances
        distances = numpy.sqrt(numpy.sum((self.X_train - x)**2, axis=1))

        # Get indices of K nearest neighbors
        # numpy.argpartition finds K smallest values efficiently
        k_indices = numpy.argpartition(distances, self.k)[:self.k].astype(int)

        # Get labels of K nearest neighbors
        k_labels = self.y_train[k_indices]

        # Count votes for each class
        # Find which class got the most votes
        unique_labels, counts = numpy.unique(k_labels, return_counts=True)
        winner = unique_labels[numpy.argmax(counts)]

        return winner

    def predict(self, X_test):
        predictions = []
        for x in X_test:
            predictions.append(self.predict_single(x))
        return numpy.array(predictions)


# Algorythm 3 Multi Layer Perceptron: A calssical neural network that trains x ammount of perceprtons over y epochs then predicts based on activated trained features
class MultiLayerPerceptron:
    def __init__(self, hidden_neurons=150, epochs=50, learning_rate=0.1, random_seed=42):
        self.hidden_neurons = hidden_neurons
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.random_seed = random_seed
        self.name = f"Multi-Layer Perceptron ({hidden_neurons} neurons, {epochs} epochs)"

        # These will be set during training
        self.weights_input_hidden = None
        self.bias_hidden = None
        self.weights_hidden_output = None
        self.bias_output = None
        self.feature_mean = None
        self.feature_std = None
        self.n_classes = None
        self.input_size = None

    def sigmoid(self, x):
        # Sigmoid activation function: squashes values to range (0, 1)
        # Formula: 1 / (1 + e^(-x))
        return 1.0 / (1.0 + numpy.exp(-numpy.clip(x, -500, 500)))

    def sigmoid_derivative(self, activated_value):
        # Derivative of sigmoid - used in backpropagation
        # Formula: sigmoid(x) * (1 - sigmoid(x))
        return activated_value * (1.0 - activated_value)

    def normalize_features(self, X):
        # Normalize features to have mean=0, std=1
        if self.feature_mean is None:
            # First time - calculate mean and std from training data
            self.feature_mean = numpy.mean(X, axis=0)
            self.feature_std = numpy.std(X, axis=0)
            # Prevent division by zero
            self.feature_std[self.feature_std < 1e-9] = 1.0

        # Apply normalization
        return (X - self.feature_mean) / self.feature_std

    def initialize_weights(self):
        # Initialize weights randomly
        numpy.random.seed(self.random_seed)
        weight_range = 0.1

        # Weights from input to hidden layer
        self.weights_input_hidden = (numpy.random.rand(self.hidden_neurons, self.input_size) * 2 - 1) * weight_range
        self.bias_hidden = (numpy.random.rand(self.hidden_neurons) * 2 - 1) * weight_range

        # Weights from hidden to output layer
        self.weights_hidden_output = (numpy.random.rand(self.n_classes, self.hidden_neurons) * 2 - 1) * weight_range
        self.bias_output = (numpy.random.rand(self.n_classes) * 2 - 1) * weight_range

    def forward_pass(self, x):
        """
        Forward pass: push data through the network

        Steps:
        1. Input -> Hidden layer (with sigmoid activation)
        2. Hidden -> Output layer (with sigmoid activation)
        """
        # Input to hidden layer
        hidden_sum = numpy.dot(self.weights_input_hidden, x) + self.bias_hidden
        hidden_activation = self.sigmoid(hidden_sum)

        # Hidden to output layer
        output_sum = numpy.dot(self.weights_hidden_output, hidden_activation) + self.bias_output
        output_activation = self.sigmoid(output_sum)

        return hidden_activation, output_activation

    def train(self, X_train, y_train):
        """
        Train the neural network using backpropagation

        Backpropagation:
        1. Make a prediction (forward pass)
        2. Calculate how wrong we were (error)
        3. Adjust weights to reduce error (backward pass)
        4. Repeat many times
        """
        print(f"\nTraining {self.name}...")
        print("This may take a minute...")

        # Get dimensions
        n_samples, self.input_size = X_train.shape
        self.n_classes = len(numpy.unique(y_train))

        # Normalize features
        X_normalized = self.normalize_features(X_train)

        # Initialize weights
        self.initialize_weights()

        # Training loop
        for epoch in range(self.epochs):
            correct = 0

            # Go through each training example
            for i in range(n_samples):
                x = X_normalized[i]
                target_class = y_train[i]

                # FORWARD PASS: Make a prediction
                hidden, output = self.forward_pass(x)

                # Create target vector (one-hot encoding)
                # Example: if target_class=1 and n_classes=3, target=[0, 1, 0]
                target = numpy.zeros(self.n_classes)
                target[target_class] = 1.0

                # CALCULATE ERROR
                # Output layer error
                output_error = target - output
                output_delta = output_error * self.sigmoid_derivative(output)

                # Hidden layer error (backpropagate)
                hidden_error = numpy.dot(self.weights_hidden_output.T, output_delta)
                hidden_delta = hidden_error * self.sigmoid_derivative(hidden)

                # UPDATE WEIGHTS (gradient descent)
                # Hidden to output weights
                self.weights_hidden_output += self.learning_rate * numpy.outer(output_delta, hidden)
                self.bias_output += self.learning_rate * output_delta

                # Input to hidden weights
                self.weights_input_hidden += self.learning_rate * numpy.outer(hidden_delta, x)
                self.bias_hidden += self.learning_rate * hidden_delta

                # Track accuracy
                if numpy.argmax(output) == target_class:
                    correct += 1

            # Print progress every 10 epochs
            if (epoch + 1) % 10 == 0:
                accuracy = (correct / n_samples) * 100
                print(f"  Epoch {epoch + 1}/{self.epochs} - Training accuracy: {accuracy:.2f}%")

        print("Training complete!")

    def predict_single(self, x):
        _, output = self.forward_pass(x)
        return numpy.argmax(output)

    def predict(self, X_test):
        # Normalize test data using training statistics
        X_normalized = (X_test - self.feature_mean) / self.feature_std

        predictions = []
        for x in X_normalized:
            predictions.append(self.predict_single(x))

        return numpy.array(predictions)

print("models.py loaded successfully.")

In [ ]:
import pandas
import json
import os
from datetime import datetime

RESULTS_FILE = "run_history.csv"

def save_run(results: dict, notes: str = ""):
    # Append a single model evaluation result to the CSV history
    # Flatten the confusion matrix to a JSON string so it fits in one cell
    row = {key: value for key, value in results.items() if key != "confusion_matrix"}
    row["confusion_matrix"] = json.dumps(results.get("confusion_matrix", []))
    row["timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    row["notes"] = notes

    dataFrame_new = pandas.DataFrame([row])

    if os.path.exists(RESULTS_FILE):
        dataFrame_existing = pandas.read_csv(RESULTS_FILE)
        dataFrame_combined = pandas.concat([dataFrame_existing, dataFrame_new], ignore_index=True)
    else:
        dataFrame_combined = dataFrame_new

    dataFrame_combined.to_csv(RESULTS_FILE, index=False)
    print(f"Run saved to {RESULTS_FILE}")

def load_runs() -> pandas.DataFrame:
    # Load all historical runs into a DataFrame
    if not os.path.exists(RESULTS_FILE):
        print("No run history found.")
        return pandas.DataFrame()
    return pandas.read_csv(RESULTS_FILE)

print("run_tracker.py loaded successfully.")

In [ ]:
import pandas
import seaborn
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import json
import numpy

def plot_metrics_over_runs(df: pandas.DataFrame):
    # Line plot of key metrics across runs, grouped by model
    metrics = ["accuracy", "precision", "recall", "f1", "rmse", "r2"]
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    fig.suptitle("Model Metrics Across Runs", fontsize=16, fontweight="bold")

    for ax, metric in zip(axes.flatten(), metrics):
        seaborn.lineplot(
            data=df,
            x=df.index,
            y=metric,
            hue="model",
            marker="o",
            ax=ax
        )
        ax.set_title(metric.upper())
        ax.set_xlabel("Run #")
        ax.set_ylabel(metric)
        ax.legend(fontsize=7)

    plt.tight_layout()
    plt.savefig("metrics_over_runs.png", dpi=150)
    plt.show()


def plot_model_comparison(df: pandas.DataFrame):
    # Bar chart comparing latest run metrics per model
    # Take the most recent run per model
    latest = df.sort_values("timestamp").groupby("model").last().reset_index()

    # Convert precision, recall, and f1 to percentage for better comparison on the graph
    latest["precision"] = latest["precision"] * 100
    latest["recall"] = latest["recall"] * 100
    latest["f1"] = latest["f1"] * 100

    melted = latest.melt(
        id_vars="model",
        value_vars=["accuracy", "precision", "recall", "f1"],
        var_name="Metric",
        value_name="Score"
    )

    plt.figure(figsize=(12, 6))
    seaborn.barplot(data=melted, x="Metric", y="Score", hue="model", palette="Set2")
    plt.title("Latest Run: Model Comparison")
    plt.ylim(0, 105)  # Set y-limit to accommodate percentage values
    plt.legend(title="Model")
    plt.tight_layout()
    plt.savefig("model_comparison.png", dpi=150)
    plt.show()


def plot_confusion_matrices(df: pandas.DataFrame, class_names: list):
    # Plot confusion matrices for the most recent run of each model
    latest = df.sort_values("timestamp").groupby("model").last().reset_index()
    n_models = len(latest)

    fig, axes = plt.subplots(1, n_models, figsize=(6 * n_models, 5))
    if n_models == 1:
        axes = [axes]

    for ax, (_, row) in zip(axes, latest.iterrows()):
        cm = numpy.array(json.loads(row["confusion_matrix"]))
        seaborn.heatmap(
            cm,
            annot=True,
            fmt="d",
            cmap="Blues",
            xticklabels=class_names,
            yticklabels=class_names,
            ax=ax
        )
        ax.set_title(f"{row['model']}\nConfusion Matrix")
        ax.set_xlabel("Predicted")
        ax.set_ylabel("Actual")

    plt.tight_layout()
    plt.savefig("confusion_matrices.png", dpi=150)
    plt.show()


def plot_error_metrics(df: pandas.DataFrame):
    # Box plot of error metrics (MAE, RMSE) distribution per model across all runs
    melted = df.melt(
        id_vars="model",
        value_vars=["mae", "rmse"],
        var_name="Error Metric",
        value_name="Value"
    )
    plt.figure(figsize=(10, 6))
    seaborn.boxplot(data=melted, x="model", y="Value", hue="Error Metric", palette="Set1")
    plt.title("Error Metric Distribution Across Runs")
    plt.xticks(rotation=15)
    plt.tight_layout()
    plt.savefig("error_metrics.png", dpi=150)
    plt.show()

print("visualise.py loaded successfully.")

In [ ]:
import kagglehub
import numpy
from sklearn.model_selection import train_test_split

# Download the dataset
path = kagglehub.dataset_download("maryamlsgumel/drone-detection-dataset")
base_path = f"{path}/BirdVsDroneVsAirplane"

categories = {'Birds': 0, 'Drones': 1, 'Aeroplanes': 2}
class_names = list(categories.keys())

# Load Data
print("Loading data...")
data, labels = load_and_preprocess_data(base_path, categories)

# Clean NaN values
print("Cleaning NaN values...")
data = clean_nan_values(data)
labels = clean_nan_values(labels).astype(int)

# Flatten
n_samples = data.shape[0]
features = data.reshape(n_samples, -1)

# Show class distribution
unique_labels, counts = numpy.unique(labels, return_counts=True)
label_map = {0: "Birds", 1: "Drones", 2: "Aeroplanes"}
print(f"\nClass distribution:")
for label, count in zip(unique_labels, counts):
    print(f"  {label_map[label]}: {count} samples ({count/len(labels)*100:.1f}%)")

# Train / Test split
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"\nTrain/Test Split:")
print(f"  Training samples:  {len(X_train)} ({len(X_train)/len(features)*100:.1f}%)")
print(f"  Testing samples:   {len(X_test)} ({len(X_test)/len(features)*100:.1f}%)")
print(f"{'='*60}")

In [ ]:
# Visualize a random sample from the dataset
import matplotlib.pyplot as plt
import random

# Pick a random index
idx = random.randint(0, len(data) - 1)

# Display the image and its label
plt.imshow(data[idx])
plt.title(f"Label: {labels[idx]} (0:Bird, 1:Drone, 2:Plane)")
plt.axis('off')
plt.show()

In [ ]:
# Train all models, evaluate, and save runs
models = [
    EuclideanDistance(),
    KNearestNeighbors(k=5),
    MultiLayerPerceptron(hidden_neurons=150, epochs=50)
]

for model in models:
    print(f"\nTraining {model.name}...")
    model.train(X_train, y_train)
    results = evaluate_model(model, X_test, y_test, class_names)
    save_run(results, notes="baseline run")

In [ ]:
# Generate all charts from run history
dataFrame = load_runs()

if dataFrame.empty:
    print("No run history found — train some models first!")
else:
    print(f"Loaded {len(dataFrame)} run(s) from history.\n")
    plot_metrics_over_runs(dataFrame)
    plot_model_comparison(dataFrame)
    plot_confusion_matrices(dataFrame, class_names)
    plot_error_metrics(dataFrame)